In [2]:
# micromaba activate in terminal to get (base)
# prefered data manipulation tool
import pandas as pd

# sciKit-allel - parsing VCF files
import allel as allel

# os - Misc operating system interfaces
import os as os

# glob - Unix style pathname pattern expansion
import glob as glob

In [3]:
# collate all vcf files from SAAdapt_Clones_sequencing_data
vcf_files = [y for x in os.walk("/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data") for y in glob.glob(os.path.join(x[0], '*.vcf'))]

In [4]:

# create empty df for final filtered analysis to go into
master_df = pd.DataFrame()

# parse through each file in the list of files created above
for file in vcf_files:
    # print(file)
    # load row metadata into df
    vcf_meta_df = allel.vcf_to_dataframe(file, fields=[
        'variants/CHROM',
        'variants/POS',
        'variants/REF',
        'variants/ALT',
        'variants/ANN',
        'variants/INDEL',
        'variants/LOF',
        'variants/CLUSTER',
        ])
    # loading sample data into dict
    vcf_sample_data = allel.read_vcf(file, fields=[
        # 'variants/CHROM',
        # 'variants/POS',
        # 'variants/REF',
        # 'variants/ALT',
        # 'variants/ANN',
        'samples',
        # 'variants/INDEL',
        # 'variants/LOF',
        # 'variants/CLUSTER',
        'calldata/GT'
        ])
    # print(vcf_sample_data["calldata/GT"][0])


    # rename samples list
    samples_lst = vcf_sample_data['samples']
    # initialize dictionary with keys being samples, and values being empty lists
    samples_w_values_dict = {sample: [] for sample in samples_lst}
    # rename 3d list 
    rows_3d_lst = vcf_sample_data["calldata/GT"]

    # iterate through each row
    for row_index in range(len(rows_3d_lst)):
        # for each row, iterate through each cell
        for cell_index in range(len(rows_3d_lst[row_index])):
            # for each cell, add first value to the sample sharing the cell's index
            samples_w_values_dict[samples_lst[cell_index]].append(rows_3d_lst[row_index][cell_index][0].item())
    # create df from dictionary
    vcf_sample_data_df = pd.DataFrame(samples_w_values_dict)
    # combine metadata with sample data; column to column
    vcf_combined_df = pd.concat([vcf_meta_df, vcf_sample_data_df], axis=1)
    
    # create a list of clusters of interest (see 230413_Cluster_Gene_List_LG.xlsx)
    clusters_lst = ["3176","4938","74","460","616","2741","4529","6225","1295","3683","2897","2208","1507","7593","665","1901","434","1204","2798",
                    "331","4048","1994","6218","4352","6148","1191","7493","3087","2998","2300","4283","3249","1135","1063","1320","2564","2435",
                    "2317","3228","5039","2761","6385","2976","5284","1223","420","6127"]

    # filter df to only clusters of interest
    vcf_filt_comb_df = vcf_combined_df[vcf_combined_df['CLUSTER'].isin(clusters_lst)]

    # combine cluster-filtered data with (empty) master_df; row to row
    master_df = pd.concat([master_df, vcf_filt_comb_df], axis=0)


master_nonsyn_df = master_df[master_df['ANN'].str.contains('synonymous_variant') == False]
master_nonsyn_df.CLUSTER.unique()

# create csv file for better viewing
# master_nonsyn_df.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/master_nonsyn_df.tsv', sep='\t', index=False)
# master_df is a table containing all isolates/samples that have mutations (syn and non-syn) that correspond to our list of clusters of interest


### NOTES ON DATA STRUCTURE ####
# samples = [ samp1, samp2, samp3 ...] # access samp1 we would use samples[0]
# gt_list_of_lists = [ every row [ individual row [ individual cell ]]] # samp1 value for first row would be gt_list_of_lists[0][0][0]
# sorted(callset.keys())

### NOTES FROM SciKit-allel DOCUMENTATION ###
# 'calldata/*', 'variants/*' '*'
# 'calldata/GT','calldata/PL','samples','variants/ALT','variants/ANN','variants/CHROM','variants/CLUSTER','variants/INDEL','variantes/LOF','variants/NMD','variants/POS','variants/REF','variants/altlen','variants/is_snp','variants/numalt'


/Users/laurengonsalves/micromamba/lib/python3.12/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version=3>\t\t\t\t\t\t\t\t\t\t\t\t\t\n'
  warnings.warn('invalid INFO header: %r' % header)
/Users/laurengonsalves/micromamba/lib/python3.12/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version=3>\t\t\t\t\t\t\t\t\t\n'
  warnings.warn('invalid INFO header: %r' % header)
/Users/laurengonsalves/micromamba/lib/python3.12/site-packages/allel/io/vcf_read.py:1732: UserWarning: invalid INFO header: '##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version=3>\t\t\n'
  war

array(['2897', '4938', '4048', '1135', '4352', '74', '2798', '2208',
       '665', '2998', '1994', '4283', '434', '2564', '1223', '3176',
       '6148', '460', '2435', '7493', '420', '3683', '6225', '1507',
       '7593', '2317', '3228', '331', '1063', '2741', '1320', '1204',
       '6385', '616', '5284', '1901', '2300', '1191', '2761', '4529',
       '3249', '1295', '5039', '2976', '6127', '6218'], dtype=object)

In [5]:
# determine column heading to be used:
# list(master_nonsyn_df.columns)
# melt dataframe so there is one obsevation (variant allele) per row --> this means that there will be multiple rows per cluster and per isolate/sample, each correspoinding to a variant allele
melt_master_df = pd.melt(master_nonsyn_df, id_vars=['CHROM','POS', 'REF','ALT_1','ALT_2', 'ALT_3', 'ANN', 'INDEL', 'LOF','CLUSTER'], var_name='isolate', value_name='vcf_gt') # vcf_gt = genotype of allele (NaN, -1, 1, 0)
# melt_master_df['isolate'].nunique() # to determine total number of isolates, should matach the isoalte metadata file and the number of columns in master_nonsyn_df (minus 10 columns)

# import file with isolate metadata as pandas df
isolate_meta_df = pd.read_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/sascv_database_extract.csv')
# merge with dabase info (metadata that contains, subject, isolae, aux, etc)
merged_melt_master_df = pd.merge(melt_master_df, isolate_meta_df, on='isolate')
#merged_melt_master_df['isolate'].nunique()

# empty df for isolates with mutations in thyA
master_thyA_df = pd.DataFrame()
# extract rows that contain cluster (thyA; thymidylate synthase)
master_thyA_df = merged_melt_master_df[merged_melt_master_df['CLUSTER'].str.contains('2998')]
# master_thyA_df.dtypes # determine type of values within each column
master_thyA_df.vcf_gt.unique() # list all unique values in the column
# create a list of items that we are going to use as indicators for row removal (NaN = empty cell, 0 = same as ref allele, -1 = missing/unknown value)
rm_vcf_gt = [0., -1.]
# remove isolates that dont have an alternative allele/genotype to the reference (have a NaN, 0 or -1 in 'values' column); the '~' allow us to only pull out/save row that do not match the conditions set
master_thyA_clean_df = master_thyA_df[~(master_thyA_df.vcf_gt.isin(rm_vcf_gt)) & (master_thyA_df.vcf_gt.notnull())]
# master_thyA_clean_df['vcf_gt'].unique() # should be only one unique number (= 1 or 2, different allele from ref; the 2 indicates the second allele, listed in ALT_2 column)
# array of unique identifiers in subject column --> all subjects that had isolates with thyA mutations
master_thyA_clean_df.subject.unique()



# list of study subjects that had isolates containing thyA mutations
subj_thyA = ['12', 'AMT0047', 'AMT0506', 'AMT0172', 'AMT0204', '47', '115',
       '436', '37', 'AMT0125', 'AMT0086', 'AMT0157', 'AMT0167', 'AMT0178',
       'AMT0134', 'AMT0156', '116', 'AMT0011', 'AMT0341', 'AMT0456', '33',
       '243', 'AMT0040', 'AMT0138', 'AMT0168', 'AMT0188', 'AMT0092',
       '102', 'AMT0413', 'AMT0152', 'AMT0098', '40', 'AMT0140', 'AMT0139',
       '320', '441', 'AMT0463', '20', 'AMT0114', '415', 'AMT0115', '21',
       '108', 'AMT0177', 'AMT0099', '414', 'AMT0525', 'AMT0208', '450',
       'AMT0151', '330', 'AMT0123', '424', 'AMT0205', '333', '3', '231',
       'AMT0200', 'AMT0197', '203', '230', 'AMT0495'] # 62 subjects
# use list above to filter out subjects that have not had a thyA mutation-containing isolate (this df will contain all
# isolates from patients that have had at least one isoalte with a thyA mutation as an indicator to SXT use)
master_thyA_filtSubs_df = merged_melt_master_df[(merged_melt_master_df.subject.isin(subj_thyA)) == True]
# set(subj_thyA) ==set(master_thyA_filtSubs_df.subject.unique()) # check if the input list of subjects matches those listed in master_thyA_filtsubs_df

# list of all isoaltes from study subjects that had isoaltes containing thyA mutations
master_thyA_filtSubs_df.isolate.unique()
isol_subj_thyA = ['14-337-001422-S1', '14-337-001422-S2', '16-308-001617-S1',
       '16-308-001617-S2', '16-308-001617-S3', 'AMT0047-34', 'AMT0047-35',
       'AMT0047-37', 'AMT0047-40', 'AMT0047-42', 'AMT0047-43',
       'AMT0047-44', 'AMT0047-46', 'AMT0047-47', 'AMT0047-48',
       'AMT0047-50', 'AMT0047-51', 'AMT0047-52', 'AMT0047-53',
       'AMT0047-61', 'AMT0047-62', 'AMT0506-1', 'AMT0506-2', 'AMT0506-3',
       'AMT0506-4', 'AMT0506-5', 'AMT0506-6', 'AMT0172-11', 'AMT0172-13',
       'AMT0172-14', 'AMT0172-15', 'AMT0172-16', 'AMT0172-17',
       'AMT0172-18', 'AMT0172-19', 'AMT0172-20', 'AMT0172-21',
       'AMT0172-22', 'AMT0172-24', 'AMT0172-27', 'AMT0204-10',
       'AMT0204-11A', 'AMT0204-12', 'AMT0204-15', 'AMT0204-16',
       'AMT0204-17', 'AMT0204-1', 'AMT0204-21', 'AMT0204-3', 'AMT0204-5',
       'AMT0204-6', 'AMT0204-7', 'AMT0204-14', 'AMT0204-18', 'AMT0204-19',
       'AMT0204-2', 'AMT0204-4', '15-049-001416-S1', '16-341-001840-S1',
       '16-341-001840-S2', 'AMT0205-4', 'AMT0205-6', 'AMT0086-13',
       'AMT0086-17', 'AMT0086-26', '15-142-000601-S1', '15-209-000712-S1',
       '15-321-001053-S1', '16-022-000749-S1', '16-110-000807-S1',
       '16-162-001024-S1', '16-257-000927-S1', '15-076-001401-S1',
       '16-320-001640-S1', '16-320-001640-S2', '17-032-001420-S1',
       '15-034-001538-S1', '15-034-001538-S2', '15-097-001298-S1',
       '15-188-001565-S1', '15-188-001565-S2', '16-019-001598-S1',
       '16-019-001598-S2', '16-110-001331-S1', '16-110-001331-S2',
       '16-110-001331-S3A', '16-236-001358-S1', '16-236-001358-S2',
       '16-327-001633-S1C', 'AMT0188-16', 'AMT0188-19', 'AMT0188-1',
       'AMT0188-7', 'AMT0188-9', 'AMT0125-11', 'AMT0125-13', 'AMT0125-2',
       'AMT0125-3', 'AMT0125-4', 'AMT0125-5', 'AMT0125-6', 'AMT0125-7',
       'AMT0125-8', 'AMT0086-10', 'AMT0086-11', 'AMT0086-12',
       'AMT0086-16', 'AMT0086-18', 'AMT0086-19', 'AMT0086-22',
       'AMT0086-23', 'AMT0086-25', 'AMT0086-4', 'AMT0086-5', 'AMT0086-6',
       'AMT0086-7', 'AMT0086-8', 'AMT0086-9', 'AMT0157-13', 'AMT0157-1',
       'AMT0157-3', 'AMT0157-4', 'AMT0157-5', 'AMT0157-6', 'AMT0157-7',
       'AMT0157-8', 'AMT0167-16', 'AMT0167-17', 'AMT0167-20',
       'AMT0167-24', 'AMT0167-27', 'AMT0167-9', 'AMT0178-1', 'AMT0178-2',
       'AMT0178-3', 'AMT0178-4', 'AMT0178-5', 'AMT0178-6', 'AMT0178-7',
       'AMT0178-8', 'AMT0168-10', 'AMT0168-1', 'AMT0168-2', 'AMT0168-3',
       'AMT0168-4', 'AMT0168-6', 'AMT0168-7', 'AMT0168-8', 'AMT0168-9',
       'AMT0134-19', 'AMT0134-21', 'AMT0134-22', 'AMT0134-24',
       'AMT0134-25', 'AMT0134-26', 'AMT0134-27', 'AMT0134-28',
       'AMT0134-5', '14-316-001386-S2', '14-316-001386-S3',
       '16-250-001561-S1', 'AMT0156-10', 'AMT0156-11', 'AMT0156-12',
       'AMT0156-14', 'AMT0156-15', 'AMT0156-16', 'AMT0156-17',
       'AMT0156-18', 'AMT0156-1', 'AMT0156-2', 'AMT0156-3', 'AMT0156-4',
       'AMT0156-5', 'AMT0156-7', 'AMT0156-8', 'AMT0156-9',
       '15-023-000700-S1', '15-023-000700-S2', '16-348-001047-S1',
       'AMT0011-12', 'AMT0011-13', 'AMT0011-15', 'AMT0011-17',
       'AMT0011-18', 'AMT0011-19', 'AMT0011-20', 'AMT0011-21',
       'AMT0011-22', 'AMT0011-23', 'AMT0011-24', 'AMT0341-11',
       'AMT0341-12', 'AMT0341-2', 'AMT0341-3', 'AMT0341-4', 'AMT0341-6',
       'AMT0341-7', 'AMT0341-8', 'AMT0341-9', 'AMT0456-10', 'AMT0456-13',
       'AMT0456-14', 'AMT0456-16', 'AMT0456-17', 'AMT0456-18',
       'AMT0456-19', 'AMT0456-1', 'AMT0456-20', 'AMT0456-21',
       'AMT0456-22', 'AMT0456-23', 'AMT0456-25', 'AMT0456-4', 'AMT0456-8',
       '15-029-001491-S2', '16-327-001632-S1', '15-174-001337-S1',
       '15-265-001513-S1', '15-321-001490-S1', 'AMT0040-10', 'AMT0040-11',
       'AMT0040-12', 'AMT0040-13', 'AMT0040-14', 'AMT0040-15',
       'AMT0040-16', 'AMT0040-17', 'AMT0040-18', 'AMT0040-19',
       'AMT0040-20', 'AMT0040-23', 'AMT0040-24', 'AMT0040-25',
       'AMT0040-26', 'AMT0040-6', 'AMT0040-7A', 'AMT0040-8', 'AMT0040-9',
       'AMT0138-10', 'AMT0138-11', 'AMT0138-12', 'AMT0138-13',
       'AMT0138-2', 'AMT0138-3', 'AMT0138-4A', 'AMT0138-5', 'AMT0138-6',
       'AMT0138-7', 'AMT0138-9', 'AMT0168-14', 'AMT0168-15', 'AMT0168-16',
       'AMT0168-17', 'AMT0168-18', 'AMT0168-19', 'AMT0188-10',
       'AMT0188-11', 'AMT0188-12', 'AMT0188-13', 'AMT0188-15',
       'AMT0188-17', 'AMT0188-18', 'AMT0188-5', 'AMT0188-6', 'AMT0188-8',
       'AMT0157-11', 'AMT0157-12', 'AMT0157-9', 'AMT0092-10',
       'AMT0092-11', 'AMT0092-12', 'AMT0092-13A', 'AMT0092-14',
       'AMT0092-15', 'AMT0092-16', 'AMT0092-17', 'AMT0092-18',
       'AMT0092-4', 'AMT0092-5', 'AMT0092-6A', 'AMT0092-7', 'AMT0092-8',
       'AMT0092-9', 'AMT0495-10', 'AMT0495-11', 'AMT0495-12',
       'AMT0495-15', 'AMT0495-2', 'AMT0495-4', 'AMT0495-5', 'AMT0495-6',
       'AMT0168-12', 'AMT0168-13', '14-220-000881-S1', '14-280-000648-S1',
       '14-280-000648-S2', '15-097-000586-S1', '15-097-000586-S2',
       '15-167-000656-S1', '15-268-001038-S1', '15-268-001038-S2',
       '15-268-001038-S3', '16-022-000751-S1', '16-138-000715-S1',
       '16-138-000715-S2', '16-225-001108-S1', 'AMT0413-10', 'AMT0413-11',
       'AMT0413-12', 'AMT0413-13', 'AMT0413-15', 'AMT0413-16',
       'AMT0413-4', 'AMT0413-6', 'AMT0413-7', 'AMT0413-8', 'AMT0152-10',
       'AMT0152-11', 'AMT0152-12', 'AMT0152-13', 'AMT0152-1', 'AMT0152-2',
       'AMT0152-3', 'AMT0152-4', 'AMT0152-5', 'AMT0152-6', 'AMT0152-7',
       'AMT0152-8', 'AMT0152-9', 'AMT0098-14', 'AMT0098-15', 'AMT0098-16',
       'AMT0098-18', '15-035-001403-S1', '15-035-001403-S2',
       '16-355-001649-S1', 'AMT0140-25', 'AMT0140-26', 'AMT0139-10',
       'AMT0139-12', 'AMT0139-13', 'AMT0139-14', 'AMT0139-15',
       'AMT0139-17', 'AMT0139-18', 'AMT0139-3', 'AMT0139-7', 'AMT0139-8',
       '16-201-001596-S1', '16-201-001596-S2', '16-285-001590-S1',
       '16-285-001590-S2', 'AMT0172-23', 'AMT0172-25', 'AMT0172-26',
       '15-090-001294-S1', '17-005-001943-S1', '17-005-001943-S2',
       '17-080-001650-S1', 'AMT0140-13', 'AMT0140-16', 'AMT0140-17',
       'AMT0140-18', 'AMT0140-19', 'AMT0140-20', 'AMT0140-21',
       'AMT0140-22', 'AMT0140-23', 'AMT0140-24', 'AMT0140-27',
       'AMT0140-9', 'AMT0463-10', 'AMT0463-11', 'AMT0463-12',
       'AMT0463-13', 'AMT0463-3', 'AMT0463-4', 'AMT0463-5', 'AMT0463-6',
       'AMT0463-7', 'AMT0463-9', '14-351-001465-S1', '16-336-001612-S1',
       '16-336-001612-S2', 'AMT0114-10', 'AMT0114-11', 'AMT0114-12',
       'AMT0114-15', 'AMT0114-18', 'AMT0114-22', 'AMT0114-23',
       'AMT0114-24A', 'AMT0114-25A', 'AMT0114-28', 'AMT0114-32',
       'AMT0114-33', 'AMT0114-36', 'AMT0114-37', 'AMT0114-40',
       'AMT0114-42', 'AMT0114-45', 'AMT0114-46', 'AMT0114-48',
       'AMT0114-49', 'AMT0114-6', 'AMT0114-7', '15-132-001265-S1',
       '15-132-001265-S2', '15-229-001741-S1', '15-229-001741-S2',
       '15-229-001741-S3', '15-352-001591-S1', '15-352-001591-S2',
       '16-102-001644-S1', '16-102-001644-S2', '16-102-001644-S3',
       '16-102-001644-S4', '16-168-001540-S2', '16-168-001540-S3',
       '16-294-001704-S1', 'AMT0098-10', 'AMT0098-12', 'AMT0098-17',
       'AMT0098-19', 'AMT0098-20', 'AMT0098-21', 'AMT0098-22',
       'AMT0098-23', 'AMT0098-24', 'AMT0098-2', 'AMT0098-9', 'AMT0413-14',
       'AMT0413-18', 'AMT0413-19', 'AMT0413-5', 'AMT0413-9', 'AMT0115-13',
       'AMT0115-14', 'AMT0115-15', 'AMT0115-19', 'AMT0115-20',
       'AMT0115-21', 'AMT0115-23', 'AMT0115-24', 'AMT0115-25',
       'AMT0115-2', 'AMT0115-30', 'AMT0115-33', 'AMT0115-34',
       'AMT0115-36', 'AMT0115-37', 'AMT0115-38', 'AMT0115-4', 'AMT0115-6',
       '14-351-001466-S1', '15-089-001436-S1', '15-089-001436-S2',
       '15-301-001726-S1', '16-027-001628-S1', '16-027-001628-S2',
       '16-118-001442-S1', '16-118-001442-S2', '16-118-001442-S3',
       '16-216-001582-S1', '16-216-001582-S2', '16-336-001613-S1',
       '16-336-001613-S2', '14-266-000872-S1', '16-124-001049-S1',
       '16-124-001049-S2', 'AMT0177-11', 'AMT0177-14', 'AMT0177-15',
       'AMT0177-16', 'AMT0177-17', 'AMT0177-18', 'AMT0177-19',
       'AMT0177-1', 'AMT0177-20', 'AMT0177-21', 'AMT0177-22', 'AMT0177-8',
       'AMT0177-9', 'AMT0167-12', 'AMT0167-13', 'AMT0167-14',
       'AMT0167-15', 'AMT0167-18', 'AMT0167-19', 'AMT0167-1',
       'AMT0167-21', 'AMT0167-22', 'AMT0167-23', 'AMT0167-25',
       'AMT0167-8', 'AMT0099-10', 'AMT0099-11', 'AMT0099-12',
       'AMT0099-13', 'AMT0099-14', 'AMT0099-15', 'AMT0099-16',
       'AMT0099-17', 'AMT0099-18', 'AMT0099-19', 'AMT0099-20',
       'AMT0099-21', 'AMT0099-7', 'AMT0099-8', 'AMT0099-9',
       '15-229-001740-S1', '15-229-001740-S2', '15-229-001740-S3',
       '15-352-001594-S1', '15-352-001594-S2', '15-352-001594-S3',
       '16-102-001643-S1', '16-102-001643-S2', '16-102-001643-S3',
       '16-168-001541-S1', '16-168-001541-S2', '16-168-001541-S3',
       '16-294-001703-S1A', '16-294-001703-S2D', '16-294-001703-S2E',
       'AMT0525-1', 'AMT0525-2', 'AMT0525-3', 'AMT0208-10', 'AMT0208-3',
       'AMT0208-4', 'AMT0208-5', 'AMT0208-6', 'AMT0208-7', 'AMT0208-8',
       'AMT0208-9', '15-138-001683-S1', '16-350-001944-S1',
       '17-038-001539-S1', '17-117-001654-S1', '17-117-001654-S2',
       'AMT0151-12', 'AMT0151-13', 'AMT0151-14', 'AMT0151-15',
       'AMT0151-16', 'AMT0151-19', 'AMT0151-1', 'AMT0151-20',
       'AMT0151-21', 'AMT0151-22', 'AMT0151-23', 'AMT0151-24',
       'AMT0151-25', 'AMT0151-8A', 'AMT0151-9A', '14-343-001422-S1',
       '14-343-001422-S2', '16-291-001808-S1', '14-336-001355-S1',
       '15-062-001383-S1', '15-265-001510-S1', '15-265-001510-S2',
       '16-075-001618-S1', 'AMT0123-10', 'AMT0123-11', 'AMT0123-12',
       'AMT0123-13', 'AMT0123-14', 'AMT0123-15', 'AMT0123-16',
       'AMT0123-2', 'AMT0123-3', 'AMT0123-4', 'AMT0123-5', 'AMT0123-6',
       'AMT0123-7', 'AMT0123-8', 'AMT0123-9', 'AMT0168-5',
       '17-037-001393-S1', '15-079-001244-S1', 'AMT0205-7',
       '16-217-001529-S1', '14-316-001386-S1', '14-344-001479-S1',
       '16-316-001505-S1', 'AMT0200-13', 'AMT0200-1', 'AMT0341-10',
       '15-079-001242-S1', 'AMT0197-3', '15-049-001416-S2',
       '14-311-001285-S2', '16-110-000807-S2', '16-340-001656-S1',
       'AMT0341-14', 'AMT0341-5', '16-316-001501-S1', 'AMT0197-1',
       'AMT0495-16', '16-208-001530-S1', 'AMT0139-9']


## EASY UNMELT METHOD ##
# create list of isolates that come from subjects that have had TD_SCVS (determined above) and add to the metadata columns of interest
tdscv_isolates = list(master_thyA_filtSubs_df.columns[0:10]) + list(set(master_thyA_filtSubs_df.isolate))
# Pull the lists out from above and create new df
unmelt_master_thyA_filtSubs_df = master_nonsyn_df[tdscv_isolates]
# create csv file
unmelt_master_thyA_filtSubs_df.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/unmelt_master_thyA_filtSubs_df.tsv', sep='\t', index=False)


# set(isol_subj_thyA) == set(master_thyA_filtSubs_df.isolate.unique()) # check if the input list of isolates matches all those listed in master_thyA_filtsubs_df


# # remove metadata columns
# simple_master_thyA_filtSubs_df = master_thyA_filtSubs_df.drop(master_thyA_filtSubs_df.iloc[:, 12:31], inplace=False, axis=1)
# list(simple_master_thyA_filtSubs_df.columns)
# # simple_master_thyA_filtSubs_df.info()
# simple_master_thyA_filtSubs_df.isolate.nunique()
# # unmelt subject-filtered table for better viewing
# unmelt_master_thyA_filtSubs_df = simple_master_thyA_filtSubs_df.set_index([simple_master_thyA_filtSubs_df.index] + list(simple_master_thyA_filtSubs_df.columns[0:11].values))['vcf_gt'].unstack().reset_index()
# print(unmelt_master_thyA_filtSubs_df) #list(pivot_master_thyA_filtSubs_df.columns)
# # create csv file
# unmelt_master_thyA_filtSubs_df.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/unmelt_master_thyA_filtSubs_df.tsv', sep='\t', index=False)


In [6]:
# bring in cluster-geneID table
geneID_clstr_csv = pd.read_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/230413_Cluster_Gene_List_LG.csv')
geneID_clstr_csv['CLUSTER'] = geneID_clstr_csv['CLUSTER'].astype(int)
geneID_clstr_csv.dtypes

Gene name                   object
JE2 (CP000255.1) Gene ID    object
NWM Gene ID                 object
Aureowiki Protein           object
JE2 (CP020619.1)            object
NWM (CP023390.1)            object
CLUSTER                      int64
Ref Genome                  object
GeneID                      object
Protein                     object
Unnamed: 10                 object
dtype: object

In [7]:
# original, unmelted df :                                       manster_nonsyn_df
# Melted df, no metadata:                                       melt_master_df
# Melted df w/ metadata:                                        merged_melt_master_df
# Melted df containing data from just isolates with TD-SCVs:    master_thyA_df
# Melted df w/ all isol. from pt that had TDSCVS sometime:      master_thyA_filtSubs_df
# Above df, just unmelted and striped of meta                   unmelt_master_thyA_filtSubs_df

## WHAT WE WANT ##
# percentage of subjects (that had TDSCVs at some point and/or were treated with SXT; see subj_thyA) with mutations in clusters of interest (see clusters_lst)
# from master_thyA_filtSubs_df, group by both the cluster and subject ID; count the number of isolates with a variant allele (vcf_gt > 0; this removes 0 and -1)
# num of mutations in a cluster for a subject; subjects will have multiple clusters 
subj_cluster_ct = master_thyA_filtSubs_df[master_thyA_filtSubs_df['vcf_gt']>0][['CLUSTER', 'subject', 'isolate', 'vcf_gt']].groupby(['CLUSTER', 'subject'], as_index=False).count()#.reset_index().rename(columns = {0:'num_mut'}) # 62 subjects
subj_cluster_ct
# subj_cluster_ct.CLUSTER.nunique() # subj_cluster_ct.subject.nunique() # set(subj_thyA) ==set(subj_cluster_ct.subject.unique())
# for each cluster, deterimine number of subjects w/ vcf_gt/isolate count > 0 --> # of individuals with mutations in that cluster.
num_subj_clstr_ct = subj_cluster_ct[subj_cluster_ct['vcf_gt']>0][['CLUSTER', 'subject']].groupby(['CLUSTER'], as_index=False).count().rename(columns = {'subject':'pcnt_subj'})
num_subj_clstr_ct['pcnt_subj'] = (num_subj_clstr_ct['pcnt_subj'] / master_thyA_filtSubs_df['subject'].nunique()) * 100



In [8]:


## WHAT WE WANT ##
# percentage of isolates (from subject that had TDSCVs at some point and/or were treated with SXT; see isol_subj_thyA) with mutations in clusters of interest (see clusters_lst)

# from master_thyA_filtSubs_df, group by the cluster; count the number of isolates with a variant allele (vcf_gt > 0; this removes 0, -1, and NaN) (drop duplicate rows that from an isolate that had more than one mutation 
# in a certain cluster; were only looking for total number of isoaltes with mutations, not total number of mutations in a cluster)
isol_cluster_ct = master_thyA_filtSubs_df[(master_thyA_filtSubs_df['vcf_gt']>0) & (master_thyA_filtSubs_df.vcf_gt.notnull())].drop_duplicates(['isolate', 'CLUSTER'])[['CLUSTER', 'subject', 'isolate', 'vcf_gt']].groupby(['CLUSTER'], as_index=False).count() # 62 subjects, 604 total isolates
isol_cluster_ct[isol_cluster_ct['CLUSTER'] == '2998']
isol_cluster_ct_trimmed = isol_cluster_ct.drop(['subject', 'vcf_gt'], axis=1).rename(columns = {'isolate':'pcnt_isol'})
isol_cluster_ct_trimmed['pcnt_isol'] = (isol_cluster_ct_trimmed['pcnt_isol'] / master_thyA_filtSubs_df['isolate'].nunique()) * 100




In [154]:
# combine percent_subj and percent_isolate df by cluster
merged_clster_ct = pd.merge(isol_cluster_ct_trimmed, num_subj_clstr_ct, on='CLUSTER')
merged_clster_ct
merged_clster_ct['CLUSTER'] = merged_clster_ct['CLUSTER'].astype('int')


# add geneID and protein columns by matching on cluster
merged_clster_ct_gene = pd.merge(merged_clster_ct, geneID_clstr_csv, on='CLUSTER')
#merged_melt_master_df['isolate'].nunique()
merged_clster_ct_gene


merged_clster_ct_gene.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/summary_master_thyA_filtSubs_df.tsv', sep='\t', index=False)




In [46]:
# original, unmelted df :                                       manster_nonsyn_df
# Melted df, no metadata:                                       melt_master_df
# Melted df w/ metadata:                                        merged_melt_master_df
# Melted df containing data from just isolates with TD-SCVs:    master_thyA_df
# Melted df w/ all isol. from pt that had TDSCVS sometime:      master_thyA_filtSubs_df
# Above df, just unmelted and striped of meta                   unmelt_master_thyA_filtSubs_df

## What we want ##
# percent of isolates that have mutations in clusters of interest, split up into TDSCVs and Non-TDSCVs, from all subjects that have had as lest one isoalte with thyA mutation (indicitive of SXT treatment)

# empty df
thyA_only = pd.DataFrame()
# pull out rows that contain 2998 in CLUSTER column and have a vcf_gt > 0 (so a 1 or 2)
thyA_only = master_thyA_filtSubs_df[master_thyA_filtSubs_df['CLUSTER'].str.contains('2998') & (master_thyA_filtSubs_df['vcf_gt']>0)] # 604 isolates, 238 w/ thyA mutation
thyA_only.isolate.unique() # 238, only includes those with 1 or 2
# list of isolates that have a 1 or 2 in in vcf_gt (isolates positive for a thyA mutation/are TD-SCVs)
thyA_isol_lst = list(set(thyA_only.isolate.unique()))
# pull out all isolates that are in the thyA_isol_lst to get table with all cluster mutations in those isolates
isol_wthyA = master_thyA_filtSubs_df[master_thyA_filtSubs_df.isolate.isin(thyA_isol_lst) & (master_thyA_filtSubs_df['vcf_gt']>0)]
isol_wthyA.isolate.nunique() # check = 238 isolates

# table that shows % of the population (TD-SCVS) that house mutations in our clusters of interest
thyA_only_cluster_ct = isol_wthyA[(isol_wthyA['vcf_gt']>0) & (isol_wthyA.vcf_gt.notnull())].drop_duplicates(['isolate', 'CLUSTER'])[['CLUSTER', 'subject', 'isolate', 'vcf_gt']].groupby(['CLUSTER'], as_index=False).count() # X subjects, X total isolates
thyA_only_cluster_ct_trimmed = thyA_only_cluster_ct.drop(['subject', 'vcf_gt'], axis=1).rename(columns = {'isolate':'pcnt_isol'})
thyA_only_cluster_ct_trimmed['pcnt_isol'] = (thyA_only_cluster_ct_trimmed['pcnt_isol'] / isol_wthyA['isolate'].nunique()) * 100
thyA_only_cluster_ct_trimmed['CLUSTER'] = thyA_only_cluster_ct_trimmed['CLUSTER'].astype('int')
# add gene ID and protein columns
thyA_only_clster_ct_gene = pd.merge(thyA_only_cluster_ct_trimmed, geneID_clstr_csv, on='CLUSTER')
thyA_only_clster_ct_gene

thyA_only_clster_ct_gene.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/summary_thyA_only_clster_ct_gene.tsv', sep='\t', index=False)


# empty df
nonthyA_isol = pd.DataFrame()
# pull our rows with isolates not in thyA_isol_list and have a vcf_gt > 0 (so a 1 or 2)
nonthyA_isol = master_thyA_filtSubs_df[~master_thyA_filtSubs_df.isolate.isin(thyA_isol_lst) & (master_thyA_filtSubs_df['vcf_gt']>0)] # 366 total isolates, 320 with mutations
nonthyA_isol.isolate.unique() # 320 isolates, only includes those with 1 or 2
# nonthyA_isol_lst = list(set(nonthyA_isol.isolate.unique()))

nonthyA_cluster_ct = nonthyA_isol.drop_duplicates(['isolate', 'CLUSTER'])[['CLUSTER', 'subject', 'isolate', 'vcf_gt']].groupby(['CLUSTER'], as_index=False).count()
nonthyA_cluster_ct_trimmed = nonthyA_cluster_ct.drop(['subject', 'vcf_gt'], axis=1).rename(columns = {'isolate':'pcnt_isol'})
nonthyA_cluster_ct_trimmed['pcnt_isol'] = (nonthyA_cluster_ct_trimmed['pcnt_isol'] / nonthyA_isol['isolate'].nunique()) * 100
nonthyA_cluster_ct_trimmed['CLUSTER'] = nonthyA_cluster_ct_trimmed['CLUSTER'].astype('int')
# add gene ID and protein columns
nonthyA_clster_ct_gene = pd.merge(nonthyA_cluster_ct_trimmed, geneID_clstr_csv, on='CLUSTER')
nonthyA_clster_ct_gene

nonthyA_clster_ct_gene.to_csv('/Users/laurengonsalves/Library/CloudStorage/OneDrive-UW/Hoffman Lab_Work/Projects_Data/SCVSA_ROS/Sequencing_SJS/ClusterWork/SAAdapt_Clones_sequencing_data/summary_nonthyA_clster_ct_gene.tsv', sep='\t', index=False)



In [ ]:
# manual check - filter for a single cluster 
filt_clstr_chk = master_thyA_filtSubs_df[master_thyA_filtSubs_df['CLUSTER'].str.contains('2998')] # (ex 2998, thyA)
# remove lines with a vcf_gt value <1
alt_allel_clstr_chk = filt_clstr_chk[(filt_clstr_chk['vcf_gt']>0) & (filt_clstr_chk.vcf_gt.notnull())]
# count number of unique subjects for that cluster
alt_allel_clstr_chk.subject.nunique()


# manual check - filter for a single cluster 
filt_clstr_chk = master_thyA_filtSubs_df[master_thyA_filtSubs_df['CLUSTER'].str.contains('2998')] # (ex 2998, thyA)
# remove lines with a vcf_gt value <1
alt_allel_clstr_chk = filt_clstr_chk[(filt_clstr_chk['vcf_gt']>0) & (filt_clstr_chk.vcf_gt.notnull())]
# count number of unique isolates for that cluster
alt_allel_clstr_chk.isolate.nunique()